## 필요한 섹터 크롤링 하기(네이버 증권)

In [ ]:
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup as bs

import pandas as pd

def getSector(ticker):
    root = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd='

    df = pd.DataFrame([], index=ticker, columns = ['SECTOR_KOSPI', 'SECTOR_WICS', 'EPS', 'BPS', 'PER', '업종PER', 'PBR', '현금배당수익률'])
    for code in tqdm(ticker):
        url = root + code
        # print(url)

        soup = bs(requests.get(url).text)

        # kospi sector 찾기
        td_list = soup.find_all('table',{'class' : 'cmp-table'})[0].find_all('td')
        sector_data = td_list[0].find_all('dt', {'class':'line-left'})
        for data in sector_data:
            data_split = data.get_text().split(' : ')
            if len(data_split)==2:
                df.loc[code, 'SECTOR_'+data_split[0]] = data_split[1]

        # 12월 결산 데이터
        dt_list = td_list[2].find_all('dt')
        for dt in dt_list[:-1]:
            dt_split = dt.get_text().split()
            if len(dt_split)>1:
                df.loc[code,dt_split[0]] = dt_split[1]

    return df

In [ ]:
from pykrx import stock

kospi_code = stock.get_market_ticker_list(date='20231227', market='KOSPI')
kospi_code[:5]

['095570', '006840', '027410', '282330', '138930']

In [ ]:
kosdaq_code = stock.get_market_ticker_list(date = '20231227', market='KOSDAQ')

In [ ]:
getSector(['032830', '088350', '085620', '082640'])

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00,  5.96it/s]


,SECTOR_KOSPI,SECTOR_WICS,EPS,BPS,PER,업종PER,PBR,현금배당수익률
032830,보험업,생명보험,"12,499","238,297",5.49,4.93,0.29,4.37%
088350,보험업,생명보험,908,"19,150",3.04,4.93,0.14,NaN
085620,보험업,생명보험,865,"23,094",5.66,4.93,0.21,NaN
082640,보험업,생명보험,797,"20,706",5.59,4.93,0.21,NaN


In [ ]:
df_sector = getSector(kospi_code[:])

100%|██████████| 953/953 [03:21<00:00,  4.74it/s]


In [ ]:
df_sector

,SECTOR_KOSPI,SECTOR_WICS,EPS,BPS,PER,업종PER,PBR,현금배당수익률
095570,서비스업,도로와철도운송,-351,"8,694",N/A,13.71,0.56,5.50%
006840,기타금융업,화학,473,"45,838",35.94,38.63,0.37,1.18%
027410,기타금융업,화학,582,"17,204",6.52,38.63,0.22,2.90%
282330,유통업,백화점과일반상점,"11,464","60,640",11.23,16.13,2.12,3.19%
138930,기타금융업,은행,"2,163","31,588",3.27,4.53,0.22,8.83%
...,...,...,...,...,...,...,...,...
079980,화학,"섬유,의류,신발,호화품","-2,413","10,188",N/A,7.97,0.40,NaN
005010,철강금속,철강,"3,802","18,573",1.30,18.93,0.27,7.09%
000540,보험업,손해보험,"3,289","21,807",0.92,4.65,0.14,NaN
000545,보험업,손해보험,"3,289","21,807",0.92,4.65,0.14,NaN


In [ ]:
KOSPI_SECTOR=df_sector[['SECTOR_KOSPI', 'SECTOR_WICS']]

In [ ]:
KOSPI_SECTOR= KOSPI_SECTOR.reset_index()

In [ ]:
kosdaq_sector = getSector(kosdaq_code[:])

  0%|          | 1/1705 [00:00<06:10,  4.60it/s]

100%|██████████| 1705/1705 [06:10<00:00,  4.60it/s]


In [ ]:
KOSDAQ_SECTOR = kosdaq_sector[['SECTOR_KOSDAQ', 'SECTOR_WICS']]

In [ ]:
KOSDAQ_SECTOR= KOSDAQ_SECTOR.reset_index()

In [ ]:
KOSDAQ_SECTOR

,index,SECTOR_KOSDAQ,SECTOR_WICS
0,060310,"기계,장비",기계
1,054620,금융,IT서비스
2,265520,반도체,디스플레이장비및부품
3,211270,통신장비,통신장비
4,126600,화학,자동차부품
...,...,...,...
1700,024060,도매,석유와가스
1701,010240,"기계,장비",기계
1702,189980,"음식료,담배",음료
1703,037440,전문기술,건설


In [ ]:
df = pd.concat([KOSPI_SECTOR, KOSDAQ_SECTOR], axis = 0)

In [ ]:
df.columns = ['ticker', 'SECTOR_KOSPI', 'SECTOR_WICS', 'SECTOR_KOSDAQ']

In [ ]:
df

,ticker,SECTOR_KOSPI,SECTOR_WICS,SECTOR_KOSDAQ
0,095570,서비스업,도로와철도운송,NaN
1,006840,기타금융업,화학,NaN
2,027410,기타금융업,화학,NaN
3,282330,유통업,백화점과일반상점,NaN
4,138930,기타금융업,은행,NaN
...,...,...,...,...
1700,024060,NaN,석유와가스,도매
1701,010240,NaN,기계,"기계,장비"
1702,189980,NaN,음료,"음식료,담배"
1703,037440,NaN,건설,전문기술


코스닥도 같은 방법으로 진행